In [1]:
from tkinter import Tk, Canvas, BOTH
from functools import reduce
from enum import Enum
import sys
import random
import logging
import importlib
import math

importlib.reload(logging)
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.CRITICAL)
logger = logging.getLogger()

logger.debug('This message should appear on the console')
logger.info('So should this')
logger.warning('And this, too')


In [2]:
class Orientation(Enum):
    PORTRAIT = 1
    LANDSCAPE = 2

In [3]:
class Utils():
                          
    def find_gcd(a,b):
        """ The Euclidean Algorithm """
        a = abs(a)
        b = abs(b)
        while a:
                a, b = b%a, a
        return b
        
        
    def gcd_list(list):
        """ Finds the GCD of numbers in a list.
        Input: List of numbers you want to find the GCD of
            E.g. [8, 24, 12]
        Returns: GCD of all numbers
        """
        return reduce(Utils.find_gcd, list)
    
    def calc_max_area(stack_store):
        area = 0
        for stack in stack_store:
            area += (stack.w * stack.h * stack.stack_size)
        return area
    
    def calc_max_dimentions(area):
        return math.floor(math.sqrt(area))

In [4]:
class Garden():
    def __init__(self, garden_name="A garden"):
        self.name = garden_name
        self.rhu = Layer()
        self.shed = SlabStore()
        logger.info("Created %s.\n", self.name)
        
    def get_paving(self):
        logger.info("Getting all the paving slabs.")
        slab_store = self.shed
        slab_store.add_stack(16,56,56)
        slab_store.add_stack(16,56,84)
        slab_store.add_stack(16,56,28)
        slab_store.add_stack(16,56,42)
        
        slab_store.find_hcf()
        slab_store.max_area= Utils.calc_max_area(slab_store.stack_store)
        slab_store.max_dimentions = Utils.calc_max_dimentions(slab_store.max_area)
        slab_store.describe()
        logger.info("Got all the paving slabs.\n")

In [5]:
class SlabStore():
    
    def __init__(self):
        self.stack_store=[]
        self.non_empty_stacks=[]
    
    def add_stack(self, stack_size, w, h):
        num_of_stacks = len(self.stack_store)
        self.stack_store.append(SlabStack(self, num_of_stacks+1, stack_size, w, h))
        self.refresh_non_empty_stacks()
        
        
    def refresh_non_empty_stacks(self):
        self.non_empty_stacks = [ stack for stack in self.stack_store if stack.stack_size!=0]
        
        
    def find_hcf(self):
        self.edge_lengths=[]
        
        for stack in self.stack_store:
            self.edge_lengths.append(stack.w)
            self.edge_lengths.append(stack.h)
            
        self.hcf = Utils.gcd_list(self.edge_lengths)
        if self.hcf <= 4:
            logger.warning('HCF is very low, grid will be Large')
    
    def describe(self):
        for stack in self.stack_store:
            stack.describe()
        logger.info("Max area: %s, Max Dimentions: %s", self.max_area, self.max_dimentions)


    #TODO return a slab obj rather than making a slab
    def return_slab(self, slab):
        for stack in self.stack_store:
            if slab.stack_id == stack.id:
                stack.remaining()
                stack.add_slab(self, slab.width, slab.height, slab.stack_id)
                stack.remaining()
                print()

    def take_random_slab(self, stacks_tried):
        self.remaining_stacks = [stack for stack in self.non_empty_stacks if stack.id not in stacks_tried]

        if len(self.remaining_stacks)==1:
            stack = self.remaining_stacks[0]
        elif len(self.remaining_stacks)==0:
            logger.critical('RAN OUT OF SLABS SOMEHOW')
            sys.exit(0)
        else:
            stack = self.remaining_stacks[random.randint(0,len(self.remaining_stacks)-1)]
        
        logger.info("Remaining stack id's %s, Non empties: %s", \
                    [i.id for i in self.remaining_stacks],\
                    [i.id for i in self.non_empty_stacks])
        stack.picked()
        slab = stack.remove_slab(self)
        stack.remaining()
        return slab
        
        
    def take_from_stack(self, id):
        #TODO find id instead of position
        stack = self.stack_store[id-1]
        stack.picked()
        slab = stack.remove_slab(self)
        stack.remaining()
        return slab
    
    #TODO move empty stack detection out
    def take_first_slab(self):
        x = 0
        while self.stack_store[x].stack_size==0:
            x+=1
        stack = self.stack_store[x]
        stack.picked()
        slab = stack.remove_slab(self)
        stack.remaining()
        return slab
    

In [6]:
class SlabStack():
    def __init__(self, id, initial_num_of_slabs, dimentions):

        self.id = id
        self.num_of_slabs= initial_num_of_slabs
        self.slabs = []  
        
        for i in range(num_of_slabs):
            self.slabs.append(Slab(self.id, dimentions))
            
        def update_num_of_slabs(self):
             self.num_of_slabs = len(self.slabs)
                
            

In [7]:
class Slab():
    
    def __init__(self, id, dimentions, orientation = Orientation.PORTRAIT):
        self.long_side = dimentions.pop(0) if dimentions[0] >= dimentions[1] else dimentions.pop(1)
        self.short_side = dimentions.pop()
        self.orientation = orientation;
        self.width = long_side if self.orientation == Orientation.LANDSCAPE else short_side
        self.height = long_side if self.orientation == Orientation.PORTRAIT else short_side
        self.grid_location = [0,0]
        
    def rotate(self):
        if self.orientation == Orientation.PORTRAIT:
            self.orientation = Orientation.LANDSCAPE
        else: 
            self.orientation = Orientation.PORTRAIT
        self.width, self.height = self.height, self.width
        return 
    
    def add_to_stack(self, stack, id, store):
        self.slabs.append(Slab(w, h, id))
        self.update_stack_size()
        store.refresh_non_empty_stacks()
        
    def take_from_stack(self, store):
        slab = self.slabs.pop()
        self.update_stack_size()
        store.refresh_non_empty_stacks()
        return slab

    def picked(self):
        logger.info("Picked type#%s:%s slabs total", str(self.id), str(self.stack_size))
    
    def remaining(self):
        logger.info("now type#%s:%s slabs remain", str(self.id), str(self.stack_size))
        
        
    def describe(self):
        logger.info("#%s - %s slabs %sx%s", str(self.id), str(self.stack_size), str(self.long_side), str(self.short_side))
#         self.slabs[0].describe()
#         print("Slab color: "  self.stack_size)
    def describe(self):
        logger.info("Orientation",self.orientation)

In [8]:
class Patio():
    
    def __init__(self, shed, width=400, height=400, patio_name="A new patio"):
        self.name=patio_name
        self.width=width
        self.height=height
        self.hcf=shed.hcf
#         print("Patio dimentions:", width,height)
        self.start_xy = [0,0]
        self.current_xy = self.start_xy
        self.grid = Grid(shed.hcf, width, height)
        self.plan=Plan(width, height)
        
        logger.info("Created %s.\n", self.name)

        
    def set_description(self, patio_desc):
        self.description = patio_desc
    
    def get_current_xy(self):
        return self.current_xy
    
    def set_current_xy_from_grid(self, current_xy):
        self.current_xy = [i*self.hcf for i in current_xy]
    
    
    def get_description(self):
        return self.description
    
    
    def describe(self):
        logger.info(self.description)
     
    
    def lay_slabs(self, slabs):
        layer = Layer()
    
    

In [9]:
class Plan():
    def __init__(self, width=400, height=400):
        
        self.tk=Tk()
        # Set some attributes
        self.tk.title( "Garden Plan" )
        self.tk.geometry(str(width)+"x"+str(height))
        self.tk.plan_width = width
        self.tk.plan_height = height
        logger.info("Plan dimentions:%s %s", width,height)
        # Create a tkinter canvas object to draw on
        self.w = Canvas(self.tk)
        self.w.pack(fill=BOTH, expand=1)

    # return the instance of tk 
    def __get__(self, instance, owner):
        return self.tk

    # allows Plan.display() to be called triggering the tk mainloop
    def display(self):
        """
        Displays the paper object
        """
        self.tk.mainloop()
    
    def draw_slab(self, slab, patio):
        self.color=["green","red","blue","yellow"]
        xy = patio.get_current_xy()
        logger.info("xy%s", xy)
        self.x1 = xy[0]        
        self.y1 = xy[1]
        self.x2 = self.x1 + slab.width
        self.y2 = self.y1 + slab.height
        self.w.create_rectangle(self.x1, self.y1, self.x2, self.y2, fill=self.color[slab.stack_id -1])


In [10]:
class Grid():
    # Origin of grid is the top left corner, as per the tkinter library.
    
    def __init__(self, hcf, patio_width=400, patio_height=400):
        self.hcf = hcf
        self.width = int(patio_width/hcf)
        self.height = int(patio_height/hcf)
        
        logger.info("Grid dimentions:", self.width, self.height)
        self.grid = [[0] * self.width for i in range(self.height)]
        self.last_clear_xy=[0,0]

        
    def show_layout(self):
        for i in range(0,self.width):
            print(self.grid[i])
        return
    
    def translate_coords(self, current_xy, slab, corners):
#         if slab.orientation == Orientation.LANDSCAPE:
#             print()
# #             width, height = slab.height, slab.width
#         else:
        width, height = slab.width, slab.height
            
        corners.x1 = int(current_xy[0]/self.hcf) if current_xy[0] !=0 else 0 
        coy1 = int(current_xy[1]/self.hcf) if current_xy[1] !=0 else 0
        x2 = int(current_xy[0] + width)/self.hcf
        y2 = int(current_xy[1] + height)/self.hcf
        
        return x1,y1,x2,y2
    
    def lay_slab(self, current_xy, slab):
        logger.info("Trying to lay a slab")
        logger.info("CURR XY =%s", current_xy)
        self.attempt_result={"success":False, "last_clear_xy":self.last_clear_xy}
        corners={"x1":0,"y1":0,"x2":0,"y2":0}
        #transform real dimentions into collision grid
        corners = self.translate_coords(current_xy, slab, corners)
        x_blocks = int(abs(x2 - x1))
        y_blocks = int(abs(y2 - y1))
        logger.debug("%s, %s, %s, %s, %s", slab.orientation, x_blocks, y_blocks, slab.width, slab.height)
        
      
        
        #Random orientation
        if random.randint(0,1) == 1:
            slab.rotate()
            x2, y2 = x2 - x_blocks + y_blocks, y2 - y_blocks + x_blocks
            x_blocks = int(abs(x2 - x1))
            y_blocks = int(abs(y2 - y1))  
            logger.info("rotated slab initially")


        #TODO move into own placement function
        #Scan grid from last placement for a new empty spot
        while True:
            #try 1st spot, original rotation
            if self.check_area_clear(x1, y1, x_blocks, y_blocks):
                self.place_slab(x1, y1, x_blocks, y_blocks)
                self.attempt_result["success"] = True
                break
            
            #try 1st spot, after rotation
            slab.rotate()
            logger.info("rotated slab")
            x2, y2 = x2 - x_blocks + y_blocks, y2 - y_blocks + x_blocks
            x_blocks = int(abs(x2 - x1))
            y_blocks = int(abs(y2 - y1)) 
            logger.info("x blocks: %s  Y blocks: %s", x_blocks, y_blocks)
            logger.debug("%s, %s, %s, %s, %s", slab.orientation, x_blocks, y_blocks, slab.width, slab.height)


            if self.check_area_clear(x1, y1, x_blocks, y_blocks):
                self.place_slab(x1, y1, x_blocks, y_blocks)
                self.attempt_result["success"] = True
                break
            
            #turn slab to original rotation to try in a new spot
            slab.rotate()
            logger.info("reset slab rotation")
            x2, y2 = x2 - x_blocks + y_blocks, y2 - y_blocks + x_blocks
            x_blocks = int(abs(x2 - x1))
            y_blocks = int(abs(y2 - y1)) 
            logger.debug("%s, %s, %s, %s, %s", slab.orientation, x_blocks, y_blocks, slab.width, slab.height)
            #find new spot 
            x1, y1 = self.shift_start_loc(x1, y1)
            
            #check new spot a viable placement
            if(x1 >= self.width - 1) and (y1 >= self.height -1):
                logger.info("current slab will not fit in the remaining spaces in this orientation.")
                break
            elif (y1 >= self.height -1):
                logger.info("Off the bottom of the grid")
                break
            #try same slab in new spot
            logger.debug("Found new starting block.")
               
        #set placement xy
        logger.info("Next potential location: x:%s, y:%s", (self.last_clear_xy[0]+x_blocks), (self.last_clear_xy[1]+y_blocks))
        self.attempt_result["last_clear_xy"]=self.last_clear_xy
        logger.info(self.attempt_result)
#         print("need new slab please")                
        
        return self.attempt_result


    def check_area_clear(self, x1, y1, x_blocks, y_blocks):
        logger.info("Checking area clear")

        for i in range(x_blocks):
            for j in range(y_blocks):
#                 print(i,j,x1,y1)
                
                if (y1+j) >= self.height or (x1+i) >= self.width:
                    logger.info("SLAB OUT OF BOUND")
                    return False
                
                if self.grid[y1+j][x1+i] != 0:
                    return False
                
        self.last_clear_xy=[x1,y1]
        return True
    
    
    
    def shift_start_loc(self, x1, y1):
        
        while True:
            if (x1 + 1) == self.width:
                x1 = 0
                y1 +=1
                logger.debug("End of row, moving to start of next row.")
            elif (y1+1) == self.height:
#                 print("Reached end of grid, exiting. \nImplement check empty squares")
#                 sys.exit()
                logger.warning("Should not have reached this ever")
                break
            else:
                x1 += 1    
                logger.debug("%s %s Occupied, shifting right one column", x1, y1)
                
            if self.grid[y1][x1] == 0:
                break
                
        return x1, y1
    
#     def fill_gaps
    
    def place_slab(self, x1, y1, x_blocks, y_blocks):
        logger.info("laying")
        for i in range(x_blocks):
            for j in range(y_blocks):
#                 print(i,j)
                self.grid[y1+j][x1+i] = 1
#         self.show_layout()


    def find_gaps(self):
        
        for i in range(0,self.width):
            for j in range(0,self.height):
                if self.grid[j][i] ==0:
                    return True
        return False

In [11]:
class Layer():
    
    def nine_to_five(self, slab_store, patio):
        print("\nStarted laying the whole patio!!\n")
        

        while True:
            if not self.lay_single(slab_store, patio):
                print("Patio Finished")
                
                break
                
        
    def lay_single(self, slab_store, patio):
        logger.debug("Laying a single slab")
        self.slabs_tried=[]
        
        #         SLABS TRIED IN DIFFERENT ORINTATION


        while True:
            logger.info("slabs tried:%s", self.slabs_tried)
            
            if len(self.slabs_tried) == len(slab_store.non_empty_stacks):
                logger.info("TRIED ALL SLABS")
                return False
                
            self.a_slab = slab_store.take_random_slab(self.slabs_tried)
            self.slabs_tried.append(self.a_slab.stack_id)
            res = patio.grid.lay_slab(patio.get_current_xy(), self.a_slab)
            patio.set_current_xy_from_grid(res["last_clear_xy"])

            if res["success"]==False:
                logger.info("Trying another size slab")
                slab_store.return_slab(self.a_slab)
                
            else:
                patio.plan.draw_slab(self.a_slab, patio)
                logger.info("LAID SLAB")
                break
            
        
        patio.grid.show_layout()
        return True
        #add the slab to the plan
#         patio.plan

In [12]:
count = 0
while True:
    garden = Garden("Chase Back Garden")
    garden.get_paving()
    
    garden.patio = Patio(garden.shed, 392, 392)
    garden.rhu.nine_to_five(garden.shed, garden.patio)
    if not garden.patio.grid.find_gaps():
        break
    
    if count ==2:
        break
    count+=1
garden.patio.plan.display()

#TODO check enough slabs to fill grid, 
#check slabs will edge the grid without gaps
#TODO make grid smLLER

TypeError: __init__() takes 4 positional arguments but 6 were given